## Titanic Problem:
#### Learning linear regression with tensor flow and using it to predict whether a passenger of the titanic will survive

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow import feature_column as fc
import tensorflow as tf

2023-10-07 20:05:59.762562: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-07 20:05:59.789993: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
